5장의 RNN은 과거의 정보를 기억할 수 있었지만 성능이 좋지 못하다.   
주된 원인은 시계열 데이터에서 시간적으로 멀리 떨어진 장기 의존 관계를 잘 학습할 수 없다는 것.  통칭 `Long Term Dependency Problem`! 

이번 장에서는 `LSTM`이나 `GRU`와 같은 게이트가 추가된 RNN을 알아보자.

# RNN의 문제점

RNN은 시계열 데이터의 장기 의존 관계를 학습하지 못한다.    
그 원인은 BPTT<sup>Backpropagation Through Time</sup>에서 기울기 소실 `Gradient Vanishing` 혹은 기울기 폭발 `Gradient Exploding` 이 일어나기 때문!

wowwow

In [ ]:
## RNN 복습